In [27]:
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from functools import reduce

In [28]:
# load cv msle data
msles_df1 = pd.read_csv("store_nbr_family_best_msles2.csv")
msles_df2 = pd.read_csv("top_down_store_nbr_family_best_msles2.csv")
msles_df3 = pd.read_csv("top_down_family_then_store_nbr_best_msles2.csv")

cutoffs = msles_df1.columns[2:]

In [29]:
# calculate mean rmsle across folds
msles_df1["cv_rmsle"] = (msles_df1.iloc[:, 2:]**0.5).mean(axis=1)
msles_df2["cv_rmsle"] = (msles_df2.iloc[:, 3:]**0.5).mean(axis=1)
msles_df3["cv_rmsle"] = (msles_df3.iloc[:, 2:]**0.5).mean(axis=1)

msles_df1 = msles_df1.drop(cutoffs, axis=1)
msles_df2 = msles_df2.drop(cutoffs, axis=1)
msles_df3 = msles_df3.drop(cutoffs, axis=1)

In [30]:
# merge all msle_dfs
msles_df = msles_df1.merge(msles_df2, on=["store_nbr", "family"])
msles_df = msles_df.rename({"cv_rmsle_x":"cv_rmsle_1", "cv_rmsle_y":"cv_rmsle_2"}, axis=1).drop("cluster", axis=1)

msles_df = msles_df.merge(msles_df3, on=["store_nbr", "family"])
msles_df = msles_df.rename({"cv_rmsle":"cv_rmsle_3"}, axis=1)

In [31]:
# argmin of different models
msles_df["best"] = msles_df.iloc[:, -3:].idxmin(axis=1)
argmin_df = msles_df[["store_nbr", "family", "best"]]

In [32]:
msles_df.groupby("best").agg({"best":"count"})

,best
best,
cv_rmsle_1,1178
cv_rmsle_2,349
cv_rmsle_3,255


In [43]:
pred_cols = ["store_nbr", "family", "ds", "yhat"]
pred_df1 = pd.read_csv("single_level_predictions.csv", usecols=pred_cols)
pred_df2 = pd.read_csv("top_down_predictions.csv", usecols=pred_cols)
pred_df3 = pd.read_csv("family_down_predictions.csv", usecols=pred_cols)

In [44]:
pred_cols = ["store_nbr", "family", "ds", "yhat"]

pred_df1 = pred_df1.merge(argmin_df, on=["store_nbr", "family"])
pred_df1 = pred_df1[pred_df1["best"] == "cv_rmsle_1"][pred_cols]

pred_df2 = pred_df2.merge(argmin_df, on=["store_nbr", "family"])
pred_df2 = pred_df2[pred_df2["best"] == "cv_rmsle_2"][pred_cols]

pred_df3 = pred_df3.merge(argmin_df, on=["store_nbr", "family"])
pred_df3 = pred_df3[pred_df3["best"] == "cv_rmsle_3"][pred_cols]

In [51]:
pred_df = pd.concat([pred_df1, pred_df2, pred_df3], axis=0)[["ds", "store_nbr", "family", "yhat"]].rename({"ds":"date", "yhat":"sales"}, axis=1)

In [52]:
pred_df

,date,store_nbr,family,yhat
0,2017-08-16,1,AUTOMOTIVE,4.358628
16,2017-08-16,1,BABY CARE,0.000000
32,2017-08-16,1,BEAUTY,5.804745
48,2017-08-16,1,BEVERAGES,2214.814404
64,2017-08-16,1,BOOKS,0.151595
...,...,...,...,...
28447,2017-08-31,54,POULTRY,55.557453
28463,2017-08-31,54,PREPARED FOODS,83.345249
28479,2017-08-31,54,PRODUCE,523.416518
6223,2017-08-31,54,SCHOOL AND OFFICE SUPPLIES,0.000000
